# Clase Práctica #13 (Compilación)

En esta clase práctica estaremos extendiendo el lenguaje con el que trabajamos en la clase anterior. La principal adición al lenguaje es dar la posibilidad de definir tipos. Esto conlleva nuevos retos durante el chequeo semántico, pues es necesario chequear el uso consistente de los tipos. Varios factores deben ser considerados durante el chequeo, entre los que destacan la herencia y polimorfismo.

Las principales reglas del lenguaje se describen a continuación.

* Un programa consiste en una lista de definiciones de clases.
* Todas las clases se definen en el mismo espacio de nombres global.
* Cada clase tiene atributos y métodos.
* Los atributos tienen un tipo asociado.
* Los métodos tienen un tipo de retorno (que puede ser `void`), y una lista de argumentos.
* Todos los métodos son de instancia y dentro de ellos es visible `self` _(solo lectura)_, cuyo tipo estático coincide con el de la clase que implementa el método.
* Todos los atributos son privados y todos los métodos son públicos.
* Existe herencia simple.
* Un método se puede sobrescribir sí y solo sí se mantiene exactamente la misma definición para los tipos de retorno y de los argumentos.
* No existen sobrecargas de métodos ni de operadores.

La definición de las clases sigue el formato: `class <class-name> { ... }`. En la definición, los atributos y los métodos se pueden intercalar. En caso de herencia, la sintaxis cambia a: `class <class-name> : <parent-name> { ... }`.

La definición de atributos tiene la siguiente sintaxis: `<name>: <type>;`. Note que no se incluye expresión de inicialización, sino que el atributo toma el valor por defecto del tipo `<type>`.

La definición de métodos cambia a: `def <method-name>(<param0>:<type0>, ..., <paramN>:<typeN>) : <return-type> { ... }`. El cuerpo del método es una lista de expresiones terminadas por `;`, y el valor de retorno coincide con el de la última expresión (en caso de que el método sea `void` simplemente no se devuelve nada).

Se adiciona la expresión `new <type>()` que crea una nueva instancia del tipo `<type>`. Note que no se pueden pasar parámetros.

En esta extensión, la instrucción `print` se elimina, y la instrucción `let` pasa a ser una expresión. La expresión `let` tiene la sintaxis: `let <var>: <type> = <expr>` y evalúa al valor de la expresión (y con el tipo estático de la expresión). La variable `<var>` no puede estar previamente definida en ese ámbito. Se puede omitir la declaración del tipo y en tal caso la variable `<var>` ya debe estar definida y se realiza la asignación. La expresión `let` es la expresión de menor precedencia.

El resto de las expresiones del lenguaje se mantiene sin cambios.

> Recordemos que, por simplicar, tanto la declaración de funciones como el llamado a funciones reciben al menos un parámetro/argumento.

## Jerarquía del AST

Comencemos por definir los tipos de nodos que compondrán el AST. Recordemos que estos nodos deben ser capaces de atrapar toda la semántica del programa.



In [15]:
class Node:
    pass

In [16]:
class ProgramNode(Node):
    def __init__(self, declarations):
        self.declarations = declarations

class DeclarationNode(Node):
    pass
class ExpressionNode(Node):
    pass

In [17]:
class ClassDeclarationNode(DeclarationNode):
    def __init__(self, idx, features, parent=None):
        self.id = idx
        self.parent = parent
        self.features = features

class FuncDeclarationNode(DeclarationNode):
    def __init__(self, idx, params, return_type, body):
        self.id = idx
        self.params = params
        self.type = return_type
        self.body = body

class AttrDeclarationNode(DeclarationNode):
    def __init__(self, idx, typex):
        self.id = idx
        self.type = typex

In [18]:
class VarDeclarationNode(ExpressionNode):
    def __init__(self, idx, typex, expr):
        self.id = idx
        self.type = typex
        self.expr = expr

class AssignNode(ExpressionNode):
    def __init__(self, idx, expr):
        self.id = idx
        self.expr = expr

class CallNode(ExpressionNode):
    def __init__(self, obj, idx, args):
        self.obj = obj
        self.id = idx
        self.args = args

In [19]:
class AtomicNode(ExpressionNode):
    def __init__(self, lex):
        self.lex = lex

class BinaryNode(ExpressionNode):
    def __init__(self, left, right):
        self.left = left
        self.right = right

In [20]:
class ConstantNumNode(AtomicNode):
    pass
class VariableNode(AtomicNode):
    pass
class InstantiateNode(AtomicNode):
    pass
class PlusNode(BinaryNode):
    pass
class MinusNode(BinaryNode):
    pass
class StarNode(BinaryNode):
    pass
class DivNode(BinaryNode):
    pass

## Gramática

Modifiquemos la gramática de la clase anterior para incluir la nueva sintaxis del lenguaje. Procedamos en 2 pasos:
1. Construyamos las producciones de forma que se garanticen las reglas sintácticas.
2. Atributemos la gramática para construir el AST.

> Recordemos que la gramática debe ser parseable. Actualmente el parser más poderoso que tenemos es el LR(1) así que usaremos ese. Consecuentemente, los atributos de la gramática solo deben computar atributos sintetizados.

In [21]:
from cmp.pycompiler import Grammar

# grammar
G = Grammar()


# non-terminals
program = G.NonTerminal('<program>', startSymbol=True)
class_list, def_class = G.NonTerminals('<class-list> <def-class>')
feature_list, def_attr, def_func = G.NonTerminals('<feature-list> <def-attr> <def-func>')
param_list, param, expr_list = G.NonTerminals('<param-list> <param> <expr-list>')
expr, arith, term, factor, atom = G.NonTerminals('<expr> <arith> <term> <factor> <atom>')
func_call, arg_list  = G.NonTerminals('<func-call> <arg-list>')


# terminals
classx, let, defx, printx = G.Terminals('class let def print')
semi, colon, comma, dot, opar, cpar, ocur, ccur = G.Terminals('; : , . ( ) { }')
equal, plus, minus, star, div = G.Terminals('= + - * /')
idx, num, new = G.Terminals('id int new')


# productions
program %= class_list, lambda h,s: ProgramNode(s[1])

# <class-list>
class_list %= def_class, lambda h,s: [s[1]]
class_list %= def_class + class_list, lambda h,s: [s[1]] + s[2]

# <def-class>
def_class %= classx + idx + ocur + feature_list + ccur, lambda h,s: ClassDeclarationNode(s[2], s[4])
def_class %= classx + idx + colon + idx + ocur + feature_list + ccur, lambda h,s: ClassDeclarationNode(s[2], s[6], s[4])

# <feature-list>
feature_list %= G.Epsilon, lambda h,s: []
feature_list %= def_attr + feature_list, lambda h,s: [s[1]] + s[2]
feature_list %= def_func + feature_list, lambda h,s: [s[1]] + s[2]

# <def-attr>
def_attr %= idx + colon + idx + semi, lambda h,s: AttrDeclarationNode(s[1], s[3])

# <def-func>
def_func %= defx + idx + opar + param_list + cpar + colon + idx + ocur + expr_list + ccur, \
            lambda h,s: FuncDeclarationNode(s[2], s[4], s[7], s[9]) 

param_list %= param, lambda h,s: [ s[1] ]
param_list %= param + comma + param_list, lambda h,s: [ s[1] ] + s[3]

# <param>
param %= idx + colon + idx, lambda h,s: (s[1], s[3])

# <expr-list>
expr_list %= expr + semi, lambda h,s: [s[1]]
expr_list %= expr + semi + expr_list, lambda h,s: [s[1]] + s[3]

# <expr>
expr %= let + idx + colon + idx + equal + expr, lambda h,s: VarDeclarationNode(s[2], s[4], s[6])
expr %= let + idx + equal + expr, lambda h,s: AssignNode(s[2], s[4])
expr %= arith, lambda h,s: s[1]

# <arith>
arith %= arith + plus + term, lambda h,s: PlusNode(s[1], s[3])
arith %= arith + minus + term, lambda h,s: MinusNode(s[1], s[3])
arith %= term, lambda h,s: s[1]

# <term>
term %= term + star + factor, lambda h,s: StarNode(s[1], s[3])
term %= term + div + factor, lambda h,s: DivNode(s[1], s[3])
term %= factor, lambda h,s: s[1]

# <factor>
factor %= opar + expr + cpar, lambda h,s: s[2]
factor %= func_call, lambda h,s: s[1]
factor %= atom, lambda h,s: s[1]

# <atom>
atom %= num, lambda h,s: ConstantNumNode(s[1])
atom %= idx, lambda h,s: VariableNode(s[1])
atom %= new + idx + opar + cpar, lambda h,s: InstantiateNode(s[2])

# <func-call>
func_call %= factor + dot + idx + opar + arg_list + cpar, lambda h,s: CallNode(s[1], s[3], s[5])

arg_list %= expr, lambda h,s: [ s[1] ]
arg_list %= expr + comma + arg_list, lambda h,s: [ s[1] ] + s[3]

if __name__ == '__main__': 
    print(G)

Non-Terminals:
	<program>, <class-list>, <def-class>, <feature-list>, <def-attr>, <def-func>, <param-list>, <param>, <expr-list>, <expr>, <arith>, <term>, <factor>, <atom>, <func-call>, <arg-list>
Terminals:
	class, let, def, print, ;, :, ,, ., (, ), {, }, =, +, -, *, /, id, int, new
Productions:
	[<program> -> <class-list>, <class-list> -> <def-class>, <class-list> -> <def-class> <class-list>, <def-class> -> class id { <feature-list> }, <def-class> -> class id : id { <feature-list> }, <feature-list> -> e, <feature-list> -> <def-attr> <feature-list>, <feature-list> -> <def-func> <feature-list>, <def-attr> -> id : id ;, <def-func> -> def id ( <param-list> ) : id { <expr-list> }, <param-list> -> <param>, <param-list> -> <param> , <param-list>, <param> -> id : id, <expr-list> -> <expr> ;, <expr-list> -> <expr> ; <expr-list>, <expr> -> let id : id = <expr>, <expr> -> let id = <expr>, <expr> -> <arith>, <arith> -> <arith> + <term>, <arith> -> <arith> - <term>, <arith> -> <term>, <term> -> <

## Probando la gramática

Trabajaremos con el siguiente código de ejemplo:

```csharp
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}
```

Note que todos los tokens están separados por espacios. Esto nos permitirá usar un tokenizer muy simple.

In [22]:
text = '''
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}
'''

### Tokenizando

Utilizaremos la implementación de tokenizer provista en `cmp.utils`. Esta implementación asume que los lexemas se encuentran separados por `whitespaces`. Luego, forma los tokens a partir de seleccionar desde `fixed_tokens` los tokens con lexema fijo, y obtiene los tokens de lexema variable a partir la implementación del método `tokenize_text` a decorar.

In [23]:
from cmp.utils import Token, tokenizer

fixed_tokens = { t.Name: Token(t.Name, t) for t in G.terminals if t not in { idx, num }}

feature_list %= G.Epsilon, lambda h,s: []
@tokenizer(G, fixed_tokens)
def tokenize_text(token):
    lex = token.lex
    try:
        float(lex)
        return token.transform_to(num)
    except ValueError:
        return token.transform_to(idx)

if __name__ == '__main__': 
    tokens = tokenize_text(text)

def pprint_tokens(tokens):
    indent = 0
    pending = []
    for token in tokens:
        pending.append(token)
        if token.token_type in { ocur, ccur, semi }:
            if token.token_type == ccur:
                indent -= 1
            print('    '*indent + ' '.join(str(t.token_type) for t in pending))
            pending.clear()
            if token.token_type == ocur:
                indent += 1
    print(' '.join([str(t.token_type) for t in pending]))

if __name__ == '__main__':  
    pprint_tokens(tokens)

class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$


### Parseando

Comprobemos que la gramática quedó LR(1). De ser así no deberían haber conflictos shift-reduce ni reduce-reduce al construir el parser.

In [24]:
from cmp.tools.parsing import LR1Parser

if __name__ == '__main__': 
    parser = LR1Parser(G)

Además, la gramática debe haber atrapado la sintaxis del lenguaje. De ser así, el programa anterior debería poder parsearse sin problema. Comprobémoslo.

In [25]:
if __name__ == '__main__':
    parse, operations = parser([t.token_type for t in tokens], get_shift_reduce=True)
    print('\n'.join(repr(x) for x in parse))

<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<expr> -> <arith>
<expr-list> -> <expr> ;
<def-func> -> def id ( <param-list> ) : id { <expr-list> }
<def-attr> -> id : id ;
<feature-list> -> e
<feature-list> -> <def-attr> <feature-list>
<feature-list> -> <def-func> <feature-list>
<feature-list> -> <def-attr> <feature-list>
<def-class> -> class id { <feature-list> }
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> int
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<expr> -> <arith>
<expr> -> let id : id = <expr>
<atom> -> new id ( )
<factor> -> <atom>
<atom> -> int
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<expr> -> <arith>
<atom> -> id
<factor> -> <atom>
<te

### Construcción del AST

Llegados a este punto solo queda evaluar las reglas para obtener el AST. Si las reglas fueron correctamente implementadas, no deberíamos tener problemas con lo siguiente.

In [26]:
from cmp.evaluation import evaluate_reverse_parse

if __name__ == '__main__':
    ast = evaluate_reverse_parse(parse, operations, tokens)
    print(ast)

### Visitor para visualizar

Construyamos un `visitor` para visualizar el AST.

In [27]:
import cmp.visitor as visitor

class FormatVisitor(object):
    @visitor.on('node')
    def visit(self, node, tabs):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__ProgramNode [<class> ... <class>]'
        statements = '\n'.join(self.visit(child, tabs + 1) for child in node.declarations)
        return f'{ans}\n{statements}'
    
    @visitor.when(ClassDeclarationNode)
    def visit(self, node, tabs=0):
        parent = '' if node.parent is None else f": {node.parent}"
        ans = '\t' * tabs + f'\\__ClassDeclarationNode: class {node.id} {parent} {{ <feature> ... <feature> }}'
        features = '\n'.join(self.visit(child, tabs + 1) for child in node.features)
        return f'{ans}\n{features}'
    
    @visitor.when(AttrDeclarationNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__AttrDeclarationNode: {node.id} : {node.type}'
        return f'{ans}'
    
    @visitor.when(VarDeclarationNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__VarDeclarationNode: let {node.id} : {node.type} = <expr>'
        expr = self.visit(node.expr, tabs + 1)
        return f'{ans}\n{expr}'
    
    @visitor.when(AssignNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__AssignNode: let {node.id} = <expr>'
        expr = self.visit(node.expr, tabs + 1)
        return f'{ans}\n{expr}'
    
    @visitor.when(FuncDeclarationNode)
    def visit(self, node, tabs=0):
        params = ', '.join(':'.join(param) for param in node.params)
        ans = '\t' * tabs + f'\\__FuncDeclarationNode: def {node.id}({params}) : {node.type} -> <body>'
        body = '\n'.join(self.visit(child, tabs + 1) for child in node.body)
        return f'{ans}\n{body}'

    @visitor.when(BinaryNode)
    def visit(self, node, tabs=0):
        ans = '\t' * tabs + f'\\__<expr> {node.__class__.__name__} <expr>'
        left = self.visit(node.left, tabs + 1)
        right = self.visit(node.right, tabs + 1)
        return f'{ans}\n{left}\n{right}'

    @visitor.when(AtomicNode)
    def visit(self, node, tabs=0):
        return '\t' * tabs + f'\\__ {node.__class__.__name__}: {node.lex}'
    
    @visitor.when(CallNode)
    def visit(self, node, tabs=0):
        obj = self.visit(node.obj, tabs + 1)
        ans = '\t' * tabs + f'\\__CallNode: <obj>.{node.id}(<expr>, ..., <expr>)'
        args = '\n'.join(self.visit(arg, tabs + 1) for arg in node.args)
        return f'{ans}\n{obj}\n{args}'
    
    @visitor.when(InstantiateNode)
    def visit(self, node, tabs=0):
        return '\t' * tabs + f'\\__ InstantiateNode: new {node.lex}()'

Independientemente de la gramática el AST debería quedar igual. Así que la siguiente verificación no debería tener problema.

In [28]:
if __name__ == '__main__':
    formatter = FormatVisitor()
    tree = formatter.visit(ast)
    print(tree)

    assert tree == '''\__ProgramNode [<class> ... <class>]
	\__ClassDeclarationNode: class A  { <feature> ... <feature> }
		\__AttrDeclarationNode: a : int
		\__FuncDeclarationNode: def suma(a:int, b:int) : int -> <body>
			\__<expr> PlusNode <expr>
				\__ VariableNode: a
				\__ VariableNode: b
		\__AttrDeclarationNode: b : int
	\__ClassDeclarationNode: class B : A { <feature> ... <feature> }
		\__AttrDeclarationNode: c : A
		\__FuncDeclarationNode: def f(d:int, a:A) : void -> <body>
			\__VarDeclarationNode: let f : int = <expr>
				\__ ConstantNumNode: 8
			\__AssignNode: let c = <expr>
				\__CallNode: <obj>.suma(<expr>, ..., <expr>)
					\__ InstantiateNode: new A()
					\__ ConstantNumNode: 5
					\__ VariableNode: f
			\__ VariableNode: c'''

\__ProgramNode [<class> ... <class>]
	\__ClassDeclarationNode: class A  { <feature> ... <feature> }
		\__AttrDeclarationNode: a : int
		\__FuncDeclarationNode: def suma(a:int, b:int) : int -> <body>
			\__<expr> PlusNode <expr>
				\__ VariableNode: a
				\__ VariableNode: b
		\__AttrDeclarationNode: b : int
	\__ClassDeclarationNode: class B : A { <feature> ... <feature> }
		\__AttrDeclarationNode: c : A
		\__FuncDeclarationNode: def f(d:int, a:A) : void -> <body>
			\__VarDeclarationNode: let f : int = <expr>
				\__ ConstantNumNode: 8
			\__AssignNode: let c = <expr>
				\__CallNode: <obj>.suma(<expr>, ..., <expr>)
					\__ InstantiateNode: new A()
					\__ ConstantNumNode: 5
					\__ VariableNode: f
			\__ VariableNode: c


## Propuestas

- Implemente una clase contexto que le permita consultar la información de las clases, métodos y variables (especialmente los tipos).
- Implemente un visitor que recorre el AST recolectando los tipos.
- Implemente un visitor que recorre el AST recolectando los métodos y atributos.